In [1]:
import numpy as np
import pandas as pd
import os
import json
import re
os.chdir(r'\\ns-sisci01hd\home_s01\Citrix_Folder_Redirect\Downloads\NG\AutoAdjudication\mail')

In [24]:
a = '''
def feature_icd10to9_1(x):
    if x['PRI_ICD_CD'] == 'A072':
        return '74'
    elif x['PRI_ICD_CD'] == 'B2799':
        return '75'
    elif x['PRI_ICD_CD'] == 'B2792':
        return '75'
    elif x['ICD10to9_1'] != 'Derive_2':
        return x['ICD10to9_1']
    elif x['PRI_ICD_CD'] == 'A09':
        return '90'
    else:
        return '100'
'''

In [25]:
#a = open(r'\\ns-sisci01hd\home_s01\Citrix_Folder_Redirect\Downloads\NG\AutoAdjudication\mail\temp.txt').read()

to_replace = list(set(re.findall(r"(?<=\[\').*?(?=\'\])", a)))
from_replace = ["x['" + i + "']" for i in to_replace]
from_replace.append('(x)')
to_replace.append('(' + ', '.join(to_replace) + ')')

for i, j in zip(from_replace, to_replace):
    a = a.replace(i, j)
a = a.strip()

a = ' '.join(a.split())
a = a.replace(':', ':\n').replace('elif', '\nelif').replace(' if', '\n if').replace('else', '\nelse').replace(' in ', '.isin(')

In [26]:
print(a.strip())

def feature_icd10to9_1(ICD10to9_1, PRI_ICD_CD):

 if PRI_ICD_CD == 'A072':
 return '74' 
elif PRI_ICD_CD == 'B2799':
 return '75' 
elif PRI_ICD_CD == 'B2792':
 return '75' 
elif ICD10to9_1 != 'Derive_2':
 return ICD10to9_1 
elif PRI_ICD_CD == 'A09':
 return '90' 
else:
 return '100'


In [27]:
column_name = []
cond_value = []
return_value = []
if_elif = []
cond_type = []
func_def = ''
else_return = (a.split('\n')[-1]).strip()

for i in a.split('\n'):
    if 'def' in i:
        func_def = i
    if 'if ' in i:
        j = i.strip().split(' ')
        cond_type.append(j[2])
        if_elif.append(j[0])
        column_name.append(j[1])
        cond_value.append(j[3].split(':')[0])
    elif 'else:' in i:
        break
    elif 'return ' in i:
        return_value.append(i.strip().split(' ')[1])

In [29]:
print(func_def)
print(if_elif)
print(column_name)
print(cond_type)
print(cond_value)
print(return_value)
print(else_return)

def feature_icd10to9_1(ICD10to9_1, PRI_ICD_CD):
['if', 'elif', 'elif', 'elif', 'elif']
['PRI_ICD_CD', 'PRI_ICD_CD', 'PRI_ICD_CD', 'ICD10to9_1', 'PRI_ICD_CD']
['==', '==', '==', '!=', '==']
["'A072'", "'B2799'", "'B2792'", "'Derive_2'", "'A09'"]
["'74'", "'75'", "'75'", 'ICD10to9_1', "'90'"]
return '100'


In [30]:
n_column_name = [column_name[0]]
n_cond_value = [cond_value[0]]
n_return_value = [return_value[0]]
n_if_elif = [if_elif[0]]
n_cond_type = [cond_type[0]]

for i in range(1, len(if_elif)):
    if return_value[i] == return_value[i-1]\
    and column_name[i] == column_name[i-1]\
    and cond_type[i] == '=='\
    and cond_type[i-1] == '==':
        if isinstance(n_cond_value[-1], list):
            n_cond_value[-1] = n_cond_value[-1] + [cond_value[i]]
        else:
            n_cond_value[-1] = [n_cond_value[-1]] + [cond_value[i]]
    else:
        n_if_elif.append(if_elif[i])
        n_column_name.append(column_name[i])
        n_cond_type.append(cond_type[i])
        n_cond_value.append(cond_value[i])
        n_return_value.append(return_value[i])

In [51]:
print(n_if_elif)
print(n_column_name)
print(n_cond_type)
print(n_cond_value)
print(n_return_value)

['if', 'elif', 'elif', 'elif']
['PRI_ICD_CD', 'PRI_ICD_CD', 'ICD10to9_1', 'PRI_ICD_CD']
['==', '==', '!=', '==']
["'A072'", ["'B2799'", "'B2792'"], "'Derive_2'", "'A09'"]
["'74'", "'75'", 'ICD10to9_1', "'90'"]


In [32]:
fin_str = ''
for i in range(len(n_column_name)):
    if not isinstance(n_cond_value[i], list):
        fin_str += ' '.join([n_if_elif[i], n_column_name[i], '==', n_cond_value[i]])
        fin_str += ':\n return ' + n_return_value[i] + '\n'
    else:
        fin_str += n_if_elif[i] + ' ' + n_column_name[i] + ' in ' + str(n_cond_value[i]) + ':\n return ' + n_return_value[i] + '\n'
fin_str = fin_str.replace('"', '')

In [54]:
fin_str = ''
for i in range(len(n_column_name)):
    if not isinstance(n_cond_value[i], list):
        fin_str += ' '.join([n_if_elif[i], n_column_name[i], n_cond_type[i], n_cond_value[i]])
        fin_str += ':\n return ' + n_return_value[i] + '\n'
    else:
        fin_str += n_if_elif[i] + ' np.isin(' + n_column_name[i] + ', ' + str(n_cond_value[i]) + '):\n return ' + n_return_value[i] + '\n'
fin_str = fin_str.replace('"', '')

In [55]:
print(fin_str)

if PRI_ICD_CD == 'A072':
 return '74'
elif np.isin(PRI_ICD_CD, ['B2799', 'B2792']):
 return '75'
elif ICD10to9_1 != 'Derive_2':
 return ICD10to9_1
elif PRI_ICD_CD == 'A09':
 return '90'



In [56]:
a = (func_def + '\n' + fin_str.strip() + '\nelse:\n ' + else_return).strip()
func_def = a.split('\n')[0].strip()
func_name = func_def.split('def ')[1].split('(')[0]
conditions = 'conditions = ' + str(re.findall(r"(?<=if ).*?(?=:)", a))
choices = 'choices = ' + str(re.findall(r"(?<=return ).*?(?=\n)", a))
default_cond = 'default_cond = ' + a.split('else:\n')[1].split('return ')[1].strip()
returns = 'return np.select(conditions, choices, default=default_cond)'
opti_code = func_def + '\n\t' + conditions + '\n\t' + choices + '\n\t' + default_cond + '\n\t' + returns
opti_code = opti_code.replace('"', '')

In [57]:
print(opti_code)

def feature_icd10to9_1(ICD10to9_1, PRI_ICD_CD):
	conditions = [PRI_ICD_CD == 'A072', np.isin(PRI_ICD_CD, ['B2799', 'B2792']), ICD10to9_1 != 'Derive_2', PRI_ICD_CD == 'A09']
	choices = ['74', '75', 'ICD10to9_1', '90']
	default_cond = '100'
	return np.select(conditions, choices, default=default_cond)


In [182]:
a = (func_def + '\n' + fin_str.strip() + '\nelse:\n ' + else_return).strip()
func_def = a.split('\n')[0].strip()
func_name = func_def.split('def ')[1].split('(')[0]
conditions = 'conditions = ' + str(re.findall(r"(?<=if ).*?(?=:)", a))
choices = 'choices = ' + str(re.findall(r"(?<=return ).*?(?=\n)", a))
default_cond = 'default_cond = ' + a.split('else:\n')[1].split('return ')[1].strip()
returns = 'return np.select(conditions, choices, default=default_cond)'
vector_func = func_name + ' = np.vectorize(' + func_name + ')'
opti_code = func_def + '\n' + conditions + '\n' + choices + '\n' + default_cond + '\n' + returns + '\n' + vector_func
opti_code = opti_code.replace(' in ', '.isin(')
opti_code = re.sub(r"(?<=\.isin\().*?(?=])",
                   ''.join(re.findall(r"(?<=\.isin\().*?(?=\])",
                                      opti_code)) + ')',
                   opti_code).replace(')]', '])').replace('"', '').replace('\n', '\n\t')

with open(r'\\ns-sisci01hd\home_s01\Citrix_Folder_Redirect\Downloads\NG\AutoAdjudication\mail\temp_op.txt', 'w') as f:
    f.write(opti_code)

In [82]:
df = pd.DataFrame({'a': [1,2,3,4]})
df

a
0  1
1  2
2  3
3  4

In [93]:
def tt(x):
    choices = [np.all([x == 1, x != 2], axis=0)]
    conditions = ['aa']
    default_cond = x
    return np.select(choices, conditions, default_cond)
tt(df['a'])

array(['aa', '2', '3', '4'],
      dtype='<U21')

In [95]:
def tt(x):
    choices = [(x == 1) & (x != 2)]
    conditions = ['aa']
    default_cond = x
    return np.select(choices, conditions, default_cond)
tt(df['a'])

array(['aa', '2', '3', '4'],
      dtype='<U21')

In [124]:
df = pd.read_csv('H:\opti\std_dsaa_results_set_2019_09_18-06_30_14.txt', sep='|')
df = df.drop(['XeroxGroup', 'ClusterType', 'RequestReceivedTS', 'RequestReturnedTS'], axis=1)
df = df.fillna('NULLNULL')
df.head(1)

TransId  ClaimNumberCode AutoAdjudicationDecision  \
0  5c4629617c7b282333ea6aa8      51901218902                        Y   

   ApprovalRate HaveSurgeryDate HavePrognosisDate HaveHospitalizationDate  \
0         0.902               N                 N                       N   

  Microsegment ResultReasonDesc1 ResultReasonDesc2 ResultReasonDesc3  \
0          298          ICD=F339     MDC=PSYCHOSES   No Surgery Date   

   ResultReasonDesc4        ResultReasonDesc5  
0  No Prognosis Date  No Hospitalization Date

In [133]:
x = pd.read_csv('H:\opti\std_dsaa_results_set_2019_10_15-04_47_53.txt', sep='|')
x = x.drop(['XeroxGroup', 'ClusterType', 'RequestReceivedTS', 'RequestReturnedTS'], axis=1)
x = x.fillna('NULLNULL')
x.head(1)

TransId  ClaimNumberCode AutoAdjudicationDecision  \
0  5c4629617c7b282333ea6aa8      51901218902                        Y   

   ApprovalRate HaveSurgeryDate HavePrognosisDate HaveHospitalizationDate  \
0         0.902               N                 N                       N   

  Microsegment ResultReasonDesc1 ResultReasonDesc2 ResultReasonDesc3  \
0          298          ICD=F339     MDC=PSYCHOSES   No Surgery Date   

   ResultReasonDesc4        ResultReasonDesc5  
0  No Prognosis Date  No Hospitalization Date

In [134]:
(df == x).sum() / df.shape[0]

TransId                     1.0
ClaimNumberCode             1.0
AutoAdjudicationDecision    1.0
ApprovalRate                1.0
HaveSurgeryDate             1.0
HavePrognosisDate           1.0
HaveHospitalizationDate     1.0
Microsegment                1.0
ResultReasonDesc1           1.0
ResultReasonDesc2           1.0
ResultReasonDesc3           1.0
ResultReasonDesc4           1.0
ResultReasonDesc5           1.0
dtype: float64

In [135]:
df[df['Microsegment'] != x['Microsegment']].head()

Empty DataFrame
Columns: [TransId, ClaimNumberCode, AutoAdjudicationDecision, ApprovalRate, HaveSurgeryDate, HavePrognosisDate, HaveHospitalizationDate, Microsegment, ResultReasonDesc1, ResultReasonDesc2, ResultReasonDesc3, ResultReasonDesc4, ResultReasonDesc5]
Index: []

In [132]:
x[df['Microsegment'] != x['Microsegment']]['ResultReasonDesc2'].value_counts()

MDC=DIGESTIVE  SYSTEM    958
Name: ResultReasonDesc2, dtype: int64

In [75]:
a = open(r'\\ns-sisci01hd\home_s01\Citrix_Folder_Redirect\Downloads\NG\AutoAdjudication\mail\temp.txt').read()

to_replace = list(set(re.findall(r"(?<=\[\').*?(?=\'\])", a)))
from_replace = ["x['" + i + "']" for i in to_replace]
from_replace.append('(x)')
to_replace.append('(' + ', '.join(to_replace) + ')')

for i, j in zip(from_replace, to_replace):
    a = a.replace(i, j)
a = a.strip()

a = ' '.join(a.split())
a = a.replace(':', ':\n').replace('elif', '\nelif').replace(' if', '\n if').replace('else', '\nelse').replace(' in ', '.isin(')
column_name = []
cond_value = []
return_value = []
if_elif = []
cond_type = []
func_def = ''
else_return = (a.split('\n')[-1]).strip()

for i in a.split('\n'):
    if 'def' in i:
        func_def = i
    if 'if ' in i:
        j = i.strip().split(' ')
        cond_type.append(j[2])
        if_elif.append(j[0])
        column_name.append(j[1])
        cond_value.append(j[3].split(':')[0])
    elif 'else:' in i:
        break
    elif 'return ' in i:
        return_value.append(i.strip().split(' ')[1])

n_column_name = [column_name[0]]
n_cond_value = [cond_value[0]]
n_return_value = [return_value[0]]
n_if_elif = [if_elif[0]]
n_cond_type = [cond_type[0]]

for i in range(1, len(if_elif)):
    if return_value[i] == return_value[i-1]\
    and column_name[i] == column_name[i-1]\
    and cond_type[i] == '=='\
    and cond_type[i-1] == '==':
        if isinstance(n_cond_value[-1], list):
            n_cond_value[-1] = n_cond_value[-1] + [cond_value[i]]
        else:
            n_cond_value[-1] = [n_cond_value[-1]] + [cond_value[i]]
    else:
        n_if_elif.append(if_elif[i])
        n_column_name.append(column_name[i])
        n_cond_type.append(cond_type[i])
        n_cond_value.append(cond_value[i])
        n_return_value.append(return_value[i])

fin_str = ''
for i in range(len(n_column_name)):
    if not isinstance(n_cond_value[i], list):
        fin_str += ' '.join([n_if_elif[i], n_column_name[i], n_cond_type[i], n_cond_value[i]])
        fin_str += ':\n return ' + n_return_value[i] + '\n'
    else:
        fin_str += n_if_elif[i] + ' np.isin(' + n_column_name[i] + ', ' + str(n_cond_value[i]) + '):\n return ' + n_return_value[i] + '\n'
fin_str = fin_str.replace('"', '')

a = (func_def + '\n' + fin_str.strip() + '\nelse:\n ' + else_return).strip()
func_def = a.split('\n')[0].strip()
func_name = func_def.split('def ')[1].split('(')[0]
conditions = 'conditions = ' + str(re.findall(r"(?<=if ).*?(?=:)", a))
choices = 'choices = ' + str(re.findall(r"(?<=return ).*?(?=\n)", a))
default_cond = 'default_cond = ' + a.split('else:\n')[1].split('return ')[1].strip()
returns = 'return np.select(conditions, choices, default=default_cond)'
opti_code = func_def + '\n\t' + conditions + '\n\t' + choices + '\n\t' + default_cond + '\n\t' + returns
opti_code = opti_code.replace('"', '')

with open(r'\\ns-sisci01hd\home_s01\Citrix_Folder_Redirect\Downloads\NG\AutoAdjudication\mail\temp_op.txt', 'w') as f:
    f.write(opti_code)

In [81]:
def changer(func_name, df_name, cols):
    cols = cols.split(', ')
    prefix = df_name + "['"
    suffix = "'].values"
    return func_name + '(' + ', '.join([prefix + i + suffix for i in cols]) + ')'
print(changer('feature_mdc123', 'df1', 'MDC, PRI_ICD_CD, V2, V1, V3, ICD1'))

feature_mdc123(df1['MDC'].values, df1['PRI_ICD_CD'].values, df1['V2'].values, df1['V1'].values, df1['V3'].values, df1['ICD1'].values)


In [233]:
b = ''
a = "    choices = [ICD, '4', '14', '15', '20', '23', '26', '27', '29', '31', '36', '37', '42', '43', '47', '50', '64', '65', '71', '75', '85', '86', '88', '90', '92', '96', '99', '101', '103', '104', '106', '109', '111', '116', '118', '121', '122', '123', '130', '131', '133', '139', '146', '155', '158', '159', '174', '182', '184', '188', '192', '196', '198', '199', '201', '206', '222', '227', '237', '249', '251', '262', '285', '302', '333', '341', '372', '419', '526', '556', '557', '710', '724', '742', '751', '761', '773', '777', '790', '799', '801', '832', '840', '876', '904', '927', '933', '943', '961', '973', '992', '1011', '1041', '1048', '1049', '1053', '1074', '1103', '1106', '1119', '1127', '1129', '1135', '1145', '1154', '1159', '1173', '1191']"
print(len(a))
while(len(a) > 78):
    temp = (','.join(a[:78].split(',')[:-1])) + ',\n'
    b += temp
    a = a[len(temp)-1:].strip()
with open(r'\\ns-sisci01hd\home_s01\Citrix_Folder_Redirect\Downloads\NG\AutoAdjudication\mail\temp_op_pep8.txt', 'w') as f:
    f.write(b+a)

758


In [211]:
a = "    conditions = [row == 'A072', row == 'B2799', "
a.find('[')

17

In [220]:
print(a)

', row == 'B2790', row == 'B2789', row == 'B2782', row == 'B2781']


In [219]:
print(b)

                  conditions = [row == 'A072', row == 'B2799', row == 'B2792',



In [204]:
a = "   choices = [ICD10to9_1, '64681', '64681', '64681', '64681', '64681', '64681', '64681', '64681', '64681']"
a[:78]

"   choices = [ICD10to9_1, '64681', '64681', '64681', '64681', '64681', '64681'"

In [174]:
print(b+a)

    choices = [ICD10to9_1, '64681', '64681', '64681', '64681', '64681',
 '64681', '64681', '64681', '64681']


In [176]:
a[71:]

" '64681', '64681', '64681', '64681']"

In [7]:
import os
import pandas as pd
os.chdir(r'\\ns-sisci01hd\home_s01\Citrix_Folder_Redirect\Downloads\NG\AutoAdjudication')

In [19]:
f = open('std_dsaa_results_set_2019_09_19-09_14_52.txt').read()

In [21]:
f.split('\n')[1]

'5c4629617c7b282333ea6aa8|051901218902|Y|0.902|N|N|N|||298.0|ICD=F339|MDC=PSYCHOSES|No Surgery Date|No Prognosis Date|No Hospitalization Date|2019-09-19 09:13:49|2019-09-19 09:14:52'

In [22]:
df = pd.read_csv('std_dsaa_results_set_2019_09_19-09_14_52.txt', delimiter='|')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19791 entries, 0 to 19790
Data columns (total 17 columns):
TransId                     19791 non-null object
ClaimNumberCode             19791 non-null int64
AutoAdjudicationDecision    19791 non-null object
ApprovalRate                19791 non-null float64
HaveSurgeryDate             19646 non-null object
HavePrognosisDate           19646 non-null object
HaveHospitalizationDate     19646 non-null object
XeroxGroup                  0 non-null float64
ClusterType                 0 non-null float64
Microsegment                19440 non-null float64
ResultReasonDesc1           19791 non-null object
ResultReasonDesc2           19646 non-null object
ResultReasonDesc3           19646 non-null object
ResultReasonDesc4           19646 non-null object
ResultReasonDesc5           19646 non-null object
RequestReceivedTS           19791 non-null object
RequestReturnedTS           19791 non-null object
dtypes: float64(4), int64(1), object(12)
memor

In [18]:
df['PROG_DT'].dropna().head()

3937    2019-06-10
5772    2019-03-20
Name: PROG_DT, dtype: object